In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

# Data needed 

In [2]:
direc = '/ghds/pharma/datawarehouse/data/infinity_report_warehouse/'

df_all = pd.DataFrame()

#substring_list = ['202307', '202308', '202309', '202310', '202306']

for file in glob.glob(direc + '*.csv'):
    #if not any(substring in file for substring in substring_list):
        df_bip = pd.read_csv(file)
        if (len(df_bip.columns)>1):
         #if ('Sirius-1.1' in df_bip.columns[1]) :
            df = pd.read_csv(file, skiprows=1)        
            if 'Study_ID' not in df:
                df = pd.read_csv(file)
                if 'Study_ID' not in df:
                    continue
            df_all = pd.concat([df_all, df])
    
df_all.head()

/home/gyalamanchili/anaconda3/envs/jupyterlabsimplefunction2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (6,13,19,23,25,26,27,28,29,30,35,37,38,39,41,43,44,47,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/gyalamanchili/anaconda3/envs/jupyterlabsimplefunction2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (23,29,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/gyalamanchili/anaconda3/envs/jupyterlabsimplefunction2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (6,13,14,19,23,25,26,27,28,30,35,38,39,41,43,44,47,48,49,53) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/gyalamanchili/anaconda3/envs/jupyter

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,...,Cancertype,Practice_name,Physician_name,HRD_score,Amplification_type,Tumor_stage,ctDNA_detected,Somatic_variant_reportable,cfDNA_ng_yield,Alt.NoAlt
0,ARX_01,6520333569-26,A0692859,B00323252,006-006,C2D1,1.0,SUCCESS,NaN,SNV,...,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARX_01,6520333569-26,A0692859,B00323252,006-006,C2D1,1.0,SUCCESS,NaN,SNV,...,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARX_01,6520333569-26,A0692859,B00323252,006-006,C2D1,1.0,SUCCESS,NaN,SNV,...,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ARX_01,6520333569-26,A0692859,B00323252,006-006,C2D1,1.0,SUCCESS,NaN,SNV,...,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARX_01,6520333569-26,A0692859,B00323252,006-006,C2D1,1.0,SUCCESS,NaN,SNV,...,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
!pwd

/ghess/pharma/scratch/gyalamanchili/clinical_trial_data_leverage


In [4]:
clinical_trial = pd.read_csv('/ghess/pharma/scratch/gyalamanchili/clinical_trial_data_leverage/ctg-studies.csv')

# Analysis 

In [5]:
len(clinical_trial)

101863

In [6]:
study_id_of_interest = list(set(df_all['Study_ID'].dropna().astype(str)))
len(study_id_of_interest)

96

In [7]:
final_df = pd.DataFrame([])
ids_not_detected = []
for var in study_id_of_interest:
    df_needed = clinical_trial[clinical_trial['Other IDs'].fillna('-').apply(lambda x:var in x)]
    if len(df_needed)>0:
        df_needed['study_id'] = var
        final_df = pd.concat([final_df,df_needed])
        continue
    else :
        ids_not_detected.append(var)
        print (var)

AMG_07
GSK-TRACER-2
MS201924-0022
GSK EMU-019
PICI0009
IQN_01_V3


/home/gyalamanchili/anaconda3/envs/jupyterlabsimplefunction2/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


GSK- EMU- 17B
201-P2 
BOU_02
TAK-981 LBx Pilot 
ARX_01
201-2b 
GSK EMU-SOW16
MR_MATRIX
mCRPC_BIOBANK
TAK-103 LBx Pilot 2023
201-P1 C3 
TT816-12-001
201-P1 C3
JCOG1801 A1
GSK-EMU-021
IB-SASOW12_1
OSU_BR
459073
CGG21-016P
ATA188
ONO-4578-04
METRICS
ONO-4578-03
OSU_01
GSK-EMU-SOW23-2
NOPRODLUC002
CL003_1
GSK-TRACER-3
ATA129-201
BTBC
GHStudy
SOW 148
GSK-EMU-024
GSK-EMU-SOW23-1
GSK- EMU- 17A
CHIP_MATRIX
201-P1 2c
201-1 
ONO-4578-05


# Deep dive on studies we dont detect

In [35]:
# 'AMG_07' doesnt match with existing studyid, best we can do is print out Study Title, Study URL and NCT Number for a sponser
buf = clinical_trial[clinical_trial['Other IDs'].fillna('-').apply(lambda x:'AMG' in x)].copy()#[['Other IDs']]
clinical_trial[(clinical_trial.Sponsor.apply(lambda x:'amg' in x.lower())) & (clinical_trial['Brief Summary'].fillna('-').apply(lambda x:'MTAP' in x))]

,NCT Number,Study Title,Study URL,Study Status,Brief Summary,Study Results,Conditions,Interventions,Sponsor,Collaborators,Sex,Age,Study Type,Other IDs
52993,NCT05975073,A Phase 1/2 Study of AMG 193 in Combination Wi...,https://clinicaltrials.gov/study/NCT05975073,RECRUITING,The main aims of this study are to evaluate th...,NO,MTAP-null Non-Small-Cell Lung Cancer|MTAP-null...,DRUG: AMG 193|DRUG: IDE397,Amgen,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,20220127
72434,NCT05094336,"AMG 193, Methylthioadenosine (MTA) Cooperative...",https://clinicaltrials.gov/study/NCT05094336,RECRUITING,The primary objective of Parts 1 and 2 of this...,NO,Advanced MTAP-null Solid Tumors,DRUG: AMG 193|DRUG: Docetaxel|DRUG: Comparator...,Amgen,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,20210023


In [38]:
# 'GSK-TRACER-2' doesnt match with existing studyid, best we can do is print out Study Title, Study URL and NCT Number for a sponser
buf = clinical_trial[clinical_trial['Other IDs'].fillna('-').apply(lambda x:'gsk' in x)].copy()#[['Other IDs']]
#clinical_trial[(clinical_trial.Sponsor.apply(lambda x:'gsk' in x.lower())) & (clinical_trial['Brief Summary'].fillna('-').apply(lambda x:'MTAP' in x))]
buf

,NCT Number,Study Title,Study URL,Study Status,Brief Summary,Study Results,Conditions,Interventions,Sponsor,Collaborators,Sex,Age,Study Type,Other IDs
11540,NCT02980809,Topotecan Plus Apatinib Versus Topotecan Alone...,https://clinicaltrials.gov/study/NCT02980809,UNKNOWN,The purpose of this study is to determine whet...,NO,Small Cell Lung Cancer,DRUG: Apatinib|DRUG: Topotecan,First Hospitals affiliated to the China PLA Ge...,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,gsk-001


In [8]:
study_id_of_interest

['AMG_07',
 'GSK-TRACER-2',
 'ONO-4578-01',
 'MS201924-0022',
 'GSK EMU-019',
 'PICI0009',
 'RP-3500-03',
 'IQN_01_V3',
 'ACT16845',
 'MK-3475-799',
 'GSK- EMU- 17B',
 '201-P2 ',
 'BOU_02',
 'TAK-981 LBx Pilot ',
 'ARX_01',
 '201-2b ',
 '80001',
 'GSK EMU-SOW16',
 'CTMX-904-101',
 'MR_MATRIX',
 'mCRPC_BIOBANK',
 'A5481092',
 'TAK-103 LBx Pilot 2023',
 'MS201924_0001',
 '201-P1 C3 ',
 'ART4215C001',
 'C4221015',
 'M18-868',
 'TT816-12-001',
 '201-P1 C3',
 'JCOG1801 A1',
 'M21-404',
 'GSK-EMU-021',
 'ACT16444',
 'XmAb23104-01',
 'ID-VDP-103',
 'IB-SASOW12_1',
 'RP-3500-01',
 'CA116-001',
 'I3Y-MC-JPEF',
 'OSU_BR',
 '459073',
 'CGG21-016P',
 'GS-US-467-5643',
 'ATA188',
 'ONO-4578-04',
 'METRICS',
 'ONO-4578-03',
 'eFT226-0002',
 'OSU_01',
 'GSK-EMU-SOW23-2',
 'PBI-200-101',
 'NOPRODLUC002',
 'CL003_1',
 'RMC-9805-001',
 '18-301',
 'GSK-TRACER-3',
 'NADIM II',
 'ART0380C002',
 'IDE397-001',
 'D926NC00001',
 'MS201924_0020',
 'TAK-102-1501',
 '18595',
 'TTX-080-001',
 '20210023',
 'ATA129-

In [50]:
print ("Number of ids not detected :",len(ids_not_detected))

Number of ids not detected : 40


# Final dataframe

In [52]:
df_all = df_all[['Study_ID','Cancertype']].drop_duplicates(keep = 'first')
df_all['Cancertype'] = df_all['Cancertype'].fillna('-')
df_all = df_all[~df_all['Study_ID'].isna()]
df_all.index = np.arange(0,len(df_all))

In [53]:
for var in range (0,len(df_all)):
    buf = str(df_all.iloc[var]['Study_ID'])
    clinical_trial.loc[clinical_trial['Other IDs'].fillna('-').apply(lambda x:buf in x),'Study_ID'] = buf
    clinical_trial.loc[clinical_trial['Other IDs'].fillna('-').apply(lambda x:buf in x),'Cancertype'] = ', '.join(list(df_all[df_all.Study_ID == buf]['Cancertype']))

In [1]:
!pwd

/ghsfa/projects/pharma/scratch/gyalamanchili/clinical_trial_data_leverage


In [54]:
clinical_trial  = clinical_trial.rename(columns = {'Study_ID':'Study_id_Guardant_Infinity','Cancertype':'Cancertype_Guardant_Infinity'})
clinical_trial[~clinical_trial['Study_id_Guardant_Infinity'].isna()].to_csv('/ghess/pharma/scratch/gyalamanchili/clinical_trial_data_leverage/Infinity_data_clinical_trial_integration.csv',index = None)
clinical_trial[~clinical_trial['Study_id_Guardant_Infinity'].isna()]

,NCT Number,Study Title,Study URL,Study Status,Brief Summary,Study Results,Conditions,Interventions,Sponsor,Collaborators,Sex,Age,Study Type,Other IDs,Study_id_Guardant_Infinity,Cancertype_Guardant_Infinity
937,NCT04901806,Study of PBI-200 in Subjects With NTRK-Fusion-...,https://clinicaltrials.gov/study/NCT04901806,ACTIVE_NOT_RECRUITING,"This is a first-in-human, Phase 1/2 open-label...",NO,"Solid Tumor, Adult|Brain Tumor, Primary|Desmop...",DRUG: PBI-200,Pyramid Biosciences,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,PBI-200-101,PBI-200-101,-
2871,NCT03052608,A Study Of Lorlatinib Versus Crizotinib In Fir...,https://clinicaltrials.gov/study/NCT03052608,ACTIVE_NOT_RECRUITING,A phase 3 study to demonstrate whether lorlati...,YES,"Carcinoma, Non-Small-Cell Lung",DRUG: Lorlatinib|DRUG: Crizotinib,Pfizer,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,B7461006|2016-003315-35,B7461006,Lung Cancer
4940,NCT04913220,A Study of SAR444245 Combined With Cemiplimab ...,https://clinicaltrials.gov/study/NCT04913220,ACTIVE_NOT_RECRUITING,Primary Objective:\n\n-To determine the antitu...,NO,Malignant Melanoma|Squamous Cell Carcinoma of ...,DRUG: THOR-707|DRUG: Cemiplimab,Sanofi,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,ACT16845|U1111-1254-0189|2020-005332-30,ACT16845,-
6913,NCT04657068,A Study of ART0380 for the Treatment of Advanc...,https://clinicaltrials.gov/study/NCT04657068,RECRUITING,This clinical trial is evaluating a drug calle...,NO,Advanced Cancer|Metastatic Cancer|Ovarian Canc...,DRUG: ART0380|DRUG: Gemcitabine|DRUG: Irinotecan,Artios Pharma Ltd,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,ART0380C001,ART0380C001,-
10479,NCT04976803,Tissue Collection for Correlation Between ATM ...,https://clinicaltrials.gov/study/NCT04976803,COMPLETED,This study examines the correlation between AT...,NO,"Solid Tumor, Unspecified, Adult",OTHER: Determination of ATM alteration status.,Artios Pharma Ltd,NaN,ALL,"ADULT, OLDER_ADULT",OBSERVATIONAL,ART0380C002,ART0380C002,-
13464,NCT04267939,ATR Inhibitor Elimusertib (BAY1895344) Plus Ni...,https://clinicaltrials.gov/study/NCT04267939,ACTIVE_NOT_RECRUITING,The purpose of the study is to test how well p...,NO,Advanced Solid Tumors (Excluding Prostate Canc...,DRUG: Elimusertib (BAY1895344)|DRUG: Niraparib,Bayer,GlaxoSmithKline,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,18595|2018-003930-34,18595,
14419,NCT03901339,Study of Sacituzumab Govitecan-hziy Versus Tre...,https://clinicaltrials.gov/study/NCT03901339,COMPLETED,The primary objective of this study is to asse...,NO,Metastatic Breast Cancer,DRUG: Sacituzumab Govitecan-hziy|DRUG: Eribuli...,Gilead Sciences,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,IMMU-132-09|2018-004201-33,IMMU-132-09,-
15178,NCT04316039,Radiotherapy Versus Radiotherapy Combined With...,https://clinicaltrials.gov/study/NCT04316039,RECRUITING,It has been reported that radiation therapy fo...,NO,Low-grade Glioma,DRUG: Temozolomide|RADIATION: intensity modula...,West China Hospital,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,ChiCTR1800015199,80001,
16549,NCT04229979,Galinpepimut-S Versus Investigator's Choice of...,https://clinicaltrials.gov/study/NCT04229979,RECRUITING,To assess the safety and efficacy of galinpepi...,NO,Acute Myeloid Leukemia,BIOLOGICAL: Galinpepimut-S|DRUG: Azacitidine|D...,Sellas Life Sciences Group,NaN,ALL,"ADULT, OLDER_ADULT",INTERVENTIONAL,SLSG18-301,18-301,"Endometrial Carcinoma, -"
17927,NCT04421222,Oral EPI-7386 in Patients With Castration-Resi...,https://clinicaltrials.gov/study/NCT04421222,RECRUITING,"This is a phase I, clinical research study of ...",NO,Prostate Cancer,DRUG: EPI-7386 (QD)|DRUG: EPI-7386 (BID)|DRUG:...,ESSA Pharmaceuticals,NaN,MALE,"ADULT, OLDER_ADULT",INTERVENTIONAL,EPI-7386-CS-001,EPI-7386-CS-001,Prostate
